In [ ]:
from dash.dependencies import Input, Output, State
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from flask import Flask
import pandas as pd
import numpy as np
import xmltodict
import requests
import plotly
import base64
import joblib
import math
import dash
import json
import time
import xlrd
import xlwt
import io
def init_forecast():
    from plotly.subplots import make_subplots
    fig = make_subplots(
        rows=1, cols=1,
        specs=[
            [{"type": "table"}]
                ]
        )
    fig.add_trace(
        go.Table(
        columnorder = list(range(1,12,1)),
        columnwidth = [100,30,40,40,40,40,40,40,40,40,40],
        header=dict(
            values=[
            '<b>time</b>','',
            '<b>temp</b>', '<b>prec</b>', 
            '<b>RH</b>','<b>wind</b>','<b>aspect</b>','<b>gust</b>',
            '<b>high</b>', '<b>middle</b>', '<b>low</b>'
            ],
            line_color=['white','#D4F2E7','#D4F2E7','#D4F2E7','#D4F2E7','#FFE4C4','#FFE4C4','#FFE4C4','#B0C4DE','#B0C4DE','#B0C4DE'], 
            fill_color=['white','#D4F2E7','#D4F2E7','#D4F2E7','#D4F2E7','#FFE4C4','#FFE4C4','#FFE4C4','#B0C4DE','#B0C4DE','#B0C4DE'],
            align=['right','center','center','center','center','center','center','center','center','center'],
            font=dict(color='black', family='Times New Roman',size=12)
        ),
        cells=dict(
            values=[
            [],[],
                [], [],[], 
                [], [], [], 
                [], [], []
                ],
            align=['right','center','center','center','center','center','center','center','center','center'], 
            font=dict(color='black', family='Times New Roman',size=10)
            )
        ),
        row=1,col=1
    )
    fig.update_layout(
            height=100,
            width=800,
            margin=dict(
                        l=50,
                        r=50,
                        b=20,
                        t=20,
                        pad=4
                         ),
            font = dict(family='Times New Roman'),
            template='simple_white',
            showlegend=False,
            barmode='overlay')
    fig.update_xaxes(tickfont=dict(family='Times New Roman', size=1, color='black'), row=1, col=1,)
    return fig

def set_original(inputlon,inputlat,size):
    import plotly.graph_objects as go
    fig = go.Figure()
    fig = go.Figure(go.Scattermapbox(
                mode = "markers+lines",
                lat=[], lon=[]))
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(mapbox = {
            'center':go.layout.mapbox.Center(
            lat=inputlat,
            lon=inputlon),
            'style': "stamen-terrain",
            'zoom': size})
    return fig

def WGS84_to_WebM(lon,lat):
    #WGS84经纬度坐标转Web墨卡托投影坐标
    import math
    r = 20037508.34
    x = (lon * r) / 180
    y = math.log(math.tan(math.pi/4 + lat * math.pi /360))* r / math.pi
    return x,y

def resize(data,inputX,inputY,outputX,outputY):
    scale = (outputX - outputY)/(inputX - inputY)
    datao = (data - inputY) * scale + outputY
    datao = np.round(list(datao))
    datao = np.int32(datao)
    return datao

def speed_colors():
    from plotly.colors import n_colors
    import matplotlib
    d1 = n_colors((1,1,0),(0,1,0), 51)# -80 to 0
    d2 = n_colors((1,0,0),(1,1,0), 50) # 0 to 15
    d = d2+d1
    d = d[::-1]
    d = [matplotlib.colors.rgb2hex(i) for i in d]
    return d

class kml():
    def __init__(self, Input):
        from io import StringIO
        import xmltodict, json
        if isinstance(Input, str):
            with open(Input,'rb') as kml:
                obj = xmltodict.parse(kml.read())
        elif isinstance(Input,StringIO):
            kml = Input.getvalue()
            obj = xmltodict.parse(kml)
        obj = json.loads(json.dumps(obj))
        self.JSON = obj
        self.set_coordinates()
        self.set_info()
        self.calculate_range()

    def set_coordinates(self):
        import numpy as np
        import pandas as pd
        obj = self.JSON
        folder = obj['kml']['Document']['Folder']
        try:
            datafolder = folder[1]
        except:
            datafolder = folder

        if 'LineString' in datafolder['Placemark'].keys():
            coordinates = datafolder['Placemark']['LineString']['coordinates']
            coordinates = coordinates.split(sep=' ')
            coordinates = [i.split(sep=',')for i in coordinates]
            coordinates = np.array(coordinates)
            coordinates = pd.DataFrame(data=coordinates,columns=['lon','lat','ele'])
            coordinates.lon = [float(i) for i in list(coordinates.lon)]
            coordinates.lat = [float(i) for i in list(coordinates.lat)]
            coordinates.ele = [float(i) for i in list(coordinates.ele)]
            self.coordinates = coordinates
            self.type =  'raw'
        else:
            gxTrack = datafolder['Placemark']['gx:Track']
            gxCoord = gxTrack['gx:coord']
            gxWhen = gxTrack['when']
            gxExtendedData = gxTrack['ExtendedData']
            coordinates = gxCoord
            coordinates = [i.split(sep=' ')for i in coordinates]
            coordinates = np.array(coordinates)
            coordinates = pd.DataFrame(data=coordinates,columns=['lon','lat','ele'])
            coordinates['time'] = list(pd.to_datetime(gxWhen))
            d = gxExtendedData['Data']['value']
            d = [i.split(',') for i in d.split(';')]
            d = pd.DataFrame(d)
            speed = d[0]
            speed = list(speed)
            speed = speed[:-1]
            speed = [float(i) for i in speed]
            coordinates['speed'] = speed
            coordinates.lon = [float(i) for i in list(coordinates.lon)]
            coordinates.lat = [float(i) for i in list(coordinates.lat)]
            coordinates.ele = [float(i) for i in list(coordinates.ele)]
            self.type = 'detailed'
            coordinates['name'] = ''
            try:
                markers = obj['kml']['Document']['Folder'][0]['Placemark']
                for marker in markers:
                    markedTime = marker['TimeStamp']['when']
                    markedPoint = marker['Point']['coordinates']
                    markedPoint = markedPoint.split(sep=',')
                    markedPoint = [float(i) for i in markedPoint]
                    markedName = marker['name']
                    markedTime = pd.to_datetime(markedTime)
                    if coordinates[
                        (coordinates.lat == markedPoint[1]) & 
                        (coordinates.lon == markedPoint[0]) & 
                        (coordinates.ele == markedPoint[2])
                        ].shape[0] == 0:
                        markerWrite = [{'lon':markedPoint[0],
                                                    'lat':markedPoint[1],
                                                    'ele':markedPoint[2],
                                                    'time':markedTime,
                                                    'name':markedName,
                                                    'speed':0,
                                                    }]
                        coordinates = coordinates.append(markerWrite)
                    else:
                        coordinates.loc[
                        (coordinates.lat == markedPoint[1]) & 
                        (coordinates.lon == markedPoint[0]) & 
                        (coordinates.ele == markedPoint[2])
                        ,'name'] = markedName
            except:
                pass
            coordinates['time'] = pd.to_datetime(coordinates['time']).dt.time
            coordinates.sort_values('time',inplace=True)
            self.coordinates = coordinates

    def get_coordinates(self):
        return self.coordinates

    def get_json(self):
        return self.JSON

    def get_type(self):
        return self.type

    def set_info(self):
        import time

        def cal_time(ms):
            sec = int(ms)/1000
            minute = sec//60
            sec = sec%60
            hour = minute//60
            minute = minute%60
            day = hour//24
            hour = hour%240
            return [int(day),int(hour),int(minute),int(sec)]

        obj = self.JSON
        folder = obj['kml']['Document']['Folder']
        try:
            datafolder = folder[1]
        except:
            datafolder = folder
        self.documentation = obj['kml']['@xmlns']
        self.id = obj['kml']['Document']['@id']
        self.name = obj['kml']['Document']['name']
        self.description = datafolder['Placemark']['description']['div']
        self.snippet = obj['kml']['Document']['snippet']
        self.author = obj['kml']['Document']['author']
        for item in obj['kml']['Document']['ExtendedData']['Data']:
            if item['@name'] == 'TrackTags':
                self.TrackTags = item['value']
            if item['@name'] == 'BeginTime':
                self.BeginTime = time.localtime(int(str(item['value'])[:10]))
            if item['@name'] == 'EndTime':
                self.EndTime = time.localtime(int(str(item['value'])[:10]))
            if item['@name'] == 'TimeUsed':
                self.TimeUsed = cal_time(item['value'])
            if item['@name'] == 'PauseTime':
                self.TimePaused = cal_time(item['value'])
            if item['@name'] == 'PosStartName':
                self.PosStartName = item['value']
            if item['@name'] == 'PosEndName':
                self.PosEndName = item['value']
        [day_use,hour_use,minute_use,sec_use] = self.TimeUsed
        [day_pause,hour_pause,minute_pause,sec_pause] = self.TimePaused
        begintime = time.strftime('%Y-%m-%dT%H:%M:%SZ', self.BeginTime)
        self.begin = begintime
        begintime = f'Begin time: {begintime}.'
        endtime = time.strftime('%Y-%m-%dT%H:%M:%SZ', self.EndTime)
        self.end = endtime
        timeuse = f'Time used: {hour_use} hours(s), {minute_use} minute(s), {sec_use} second(s)'
        timepause = f'Time paused: {hour_pause} hours(s), {minute_pause} minute(s), {sec_pause} second(s)'
        self.use = timeuse
        self.pause = timepause

    def get_author(self):
        return self.author

    def get_trackType(self):
        return self.TrackTags

    def get_time(self):
        [day_use,hour_use,minute_use,sec_use] = self.TimeUsed
        [day_pause,hour_pause,minute_pause,sec_pause] = self.TimePaused
        begintime = time.strftime('%Y-%m-%dT%H:%M:%SZ', self.BeginTime)
        self.begin = begintime
        begintime = f'Begin time: {begintime}.'
        endtime = time.strftime('%Y-%m-%dT%H:%M:%SZ', self.EndTime)
        self.end = endtime
        endtime = f'End time: {endtime}.'
        timeuse = f'Time used: {hour_use} hours(s), {minute_use} minute(s), {sec_use} second(s).'
        timepause = f'Time paused: {hour_pause} hours(s), {minute_pause} minute(s), {sec_pause} second(s).'
        return ' '.join([begintime,endtime,timeuse,timepause])

    def get_loc(self):
        return f'start location: {self.PosStartName}, end location: {self.PosEndName}'

    def calculate_range(self):
        import math
        coordinates = self.coordinates
        (lonmin,latmin) = WGS84_to_WebM(coordinates.lon.min(),coordinates.lat.min())
        (lonmax,latmax) = WGS84_to_WebM(coordinates.lon.max(),coordinates.lat.max())
        lonrange = lonmax - lonmin
        latrange = latmax - latmin
        coordrange = max(lonrange,latrange)
        tilesize = int(math.log(40075014/coordrange,2))-1
        self.tilesize = tilesize

    def get_tilesize(self):
        return self.tilesize

    def get_customdata(self):
        if self.type == 'detailed':
            import numpy as np
            data = self.coordinates
            speesize = len(list(data.speed))
            spee = np.tile(np.array(data.speed),speesize)
            spee = np.reshape(spee, (speesize,speesize))
            spee = spee.T
            elee = np.tile(np.array(data.ele),speesize)
            elee = np.reshape(elee, (speesize,speesize))
            elee = elee.T
            return np.dstack((elee,spee))
        else:
            print("Input KML doesn't support this method.")

    def get_header(self):
        header=(f'### Track Author:  {self.author} <br> ' 
                f'###### Track Type: {self.TrackTags},{self.type}<br>' 
                f'------------------<br>' 
                f'Begin time: {self.begin}, start location: {self.PosStartName}, <br>' 
                f'<br>End time: {self.end}, end location: {self.PosEndName} <br>' 
                f'<br>Time used: {self.use}. <br>' 
                f'<br>Time paused: {self.pause}.')
        return header
        
    def set_colors(self):
        trackType = self.TrackTags
        coordinates = self.coordinates
        if trackType in ['爬山','徒步','散步']:
            maxspeed = 4
        elif trackType in ['跑步']:
            maxspeed = 10
        elif trackType in ['游泳']:
            maxspeed = 3
        elif trackType in ['骑行','滑雪']:
            maxspeed = 80
        elif trackType in ['轮滑']:
            maxspeed = 35
        elif trackType in ['摩托']:
            maxspeed = 350
        elif trackType in ['驾车']:
            maxspeed = 450
        elif trackType in ['轮船']:
            maxspeed = 55
        elif trackType in ['飞机']:
            maxspeed = 3600
        elif trackType in ['其他']:
            maxspeed = coordinates.speed.max()
        maxspeed = coordinates.speed.max()
        d = resize(coordinates.speed,maxspeed,0,100,0)
        return d

    def to_gpd(self):
        from shapely.geometry import Point
        import geopandas as gpd
        import pandas as pd
        coordinates = self.get_coordinates()
        gdf = gpd.GeoDataFrame(
            coordinates.drop(['lon', 'lat'], axis=1),
            crs={'init': 'epsg:4326'},
            geometry=[Point(xy) for xy in zip(coordinates.lon, coordinates.lat)])
        return gdf

    def write(self, filename, driverInfo, encodingInfo):
        import geopandas as gpd
        gdf = self.to_gpd()
        gpd.to_file(filename,driver=driverInfo,encoding=encodingInfo)
        
#access data through windy API 
def request(lon,lat):
    import requests
    import json
    post = {
        "lat": lat,
        "lon": lon,
        "model": "gfs",
        "parameters": ["temp", "wind", "precip", "rh", "pressure", "ptype", "lclouds", "mclouds", "hclouds", "windGust",],
        "levels": ["surface","1000h", "950h", '925h', "900h", '850h', '800h', '700h', '600h', '500h', '400h', '300h', "200h", "150h"],
        # "key": "dre0kCydf9Ce80uHxAYaQs2Vd2TCrBzV"
        "key":"voVizrMRe0hAw17zW4N4O4gkD885IenS"
        # "key": "lXIXVexIoMoYvQIjb1hoqAQmdXkLSRTb"
        # alternative 2 keys, for free API keys usage is limited.
        }
    r = requests.post('https://api.windy.com/api/point-forecast/v2', json = post)
    data = json.loads(r.text)
    return data

def getTimeSeriesVerticalWeather(JSON):
    ts = JSON['ts']
    hour=[]
    for i in range(len(ts)):
        hour.append(time.strftime("%Y-%m-%d-%Hh", time.localtime(int(str(ts[i])[:10]))))
    units = JSON['units']
    temp_units = units['temp-surface']
    uwind_units = units['wind_u-surface']
    vwind_units = units['wind_v-surface']
    past3hprecip_units = units['past3hprecip-surface']
    rh_units = units['rh-surface']
    pressure_units = units['pressure-surface']
    ptype_units = units['ptype-surface']
    clouds_units = units['lclouds-surface']
    gust_units = units['gust-surface']
    units = {'temp':temp_units,
                    'uwind':uwind_units,
                    'vwind':vwind_units,
                    'past3hprecip':past3hprecip_units,
                    'rh':rh_units,
                    'pressure':pressure_units,
                    'ptype':ptype_units,
                    'cloud':clouds_units,
                    'gust':gust_units
                    }
    temp_surface = JSON['temp-surface']
    temp_1000h = JSON['temp-1000h']
    temp_950h = JSON['temp-950h']
    temp_925h = JSON['temp-925h']
    temp_900h = JSON['temp-900h']
    temp_850h = JSON['temp-850h']
    temp_800h = JSON['temp-800h']
    temp_700h = JSON['temp-700h']
    temp_600h = JSON['temp-600h']
    temp_500h = JSON['temp-500h']
    temp_400h = JSON['temp-400h']
    temp_300h = JSON['temp-300h']
    temp_200h = JSON['temp-200h']
    temp_150h = JSON['temp-150h']
    temp = [temp_surface,temp_1000h,temp_950h,temp_925h,temp_900h,temp_850h,temp_800h,temp_700h,temp_600h,temp_500h,temp_400h,temp_300h,temp_200h,temp_150h]
    uwind_surface = JSON['wind_u-surface']
    uwind_1000h = JSON['wind_u-1000h']
    uwind_950h = JSON['wind_u-950h']
    uwind_925h = JSON['wind_u-925h']
    uwind_900h = JSON['wind_u-900h']
    uwind_850h = JSON['wind_u-850h']
    uwind_800h = JSON['wind_u-800h']
    uwind_700h = JSON['wind_u-700h']
    uwind_600h = JSON['wind_u-600h']
    uwind_500h = JSON['wind_u-500h']
    uwind_400h = JSON['wind_u-400h']
    uwind_300h = JSON['wind_u-300h']
    uwind_200h = JSON['wind_u-200h']
    uwind_150h = JSON['wind_u-150h']
    uwind = [uwind_surface,uwind_1000h,uwind_950h,uwind_925h,uwind_900h,uwind_850h,uwind_800h,uwind_700h,uwind_600h,uwind_500h,uwind_400h,uwind_300h,uwind_200h,uwind_150h]
    vwind_surface = JSON['wind_v-surface']
    vwind_1000h = JSON['wind_v-1000h']
    vwind_950h = JSON['wind_v-950h']
    vwind_925h = JSON['wind_v-925h']
    vwind_900h = JSON['wind_v-900h']
    vwind_850h = JSON['wind_v-850h']
    vwind_800h = JSON['wind_v-800h']
    vwind_700h = JSON['wind_v-700h']
    vwind_600h = JSON['wind_v-600h']
    vwind_500h = JSON['wind_v-500h']
    vwind_400h = JSON['wind_v-400h']
    vwind_300h = JSON['wind_v-300h']
    vwind_200h = JSON['wind_v-200h']
    vwind_150h = JSON['wind_v-150h']
    vwind = [vwind_surface,vwind_1000h,vwind_950h,vwind_925h,vwind_900h,vwind_850h,vwind_800h,vwind_700h,vwind_600h,vwind_500h,vwind_400h,vwind_300h,vwind_200h,vwind_150h]
    past3hprecip_surface = JSON['past3hprecip-surface']
    rh_surface = JSON['rh-surface']
    rh_1000h = JSON['rh-1000h']
    rh_950h = JSON['rh-950h']
    rh_925h = JSON['rh-925h']
    rh_900h = JSON['rh-900h']
    rh_850h = JSON['rh-850h']
    rh_800h = JSON['rh-800h']
    rh_700h = JSON['rh-700h']
    rh_600h = JSON['rh-600h']
    rh_500h = JSON['rh-500h']
    rh_400h = JSON['rh-400h']
    rh_300h = JSON['rh-300h']
    rh_200h = JSON['rh-200h']
    rh_150h = JSON['rh-150h']
    rh = [rh_surface,rh_1000h,rh_950h,rh_925h,rh_900h,rh_850h,rh_800h,rh_700h,rh_600h,rh_500h,rh_400h,rh_300h,rh_200h,rh_150h]
    pressure_surface = JSON['pressure-surface']
    ptype_surface = JSON['ptype-surface']
    lclouds_surface = JSON['lclouds-surface']
    mclouds_surface = JSON['mclouds-surface']
    hclouds_surface = JSON['hclouds-surface']
    cloud = [lclouds_surface,mclouds_surface,hclouds_surface]
    gust_surface = JSON['gust-surface']
    pressure = [0,1000,950,925,900,850,800,700,600,500,400,300,200,150]
    verticalInfo = [hour, pressure, temp, uwind, vwind, rh, cloud, pressure_surface, ptype_surface, past3hprecip_surface, gust_surface]
    return units,verticalInfo

def getCertainTimeVerticalWeather(iodata, timePoint):
    # timePoint (int): (you can use getNumberOfData(verticalInfo, timeStr) function to find index)the Index of the time you want to get the detailed weather information
    # return type: (array): [timeStr, pressure, temp, uwind, vwind, rh, cloud, pressure_surface, ptype_surface, past3hprecip_surface, gust_surface]
    if timePoint < len(iodata[0]) and timePoint >= 0:
        timeStr = iodata[0][timePoint]
        pressure = iodata[1]
        temp = [] 
        uwind = [] 
        vwind = [] 
        rh = [] 
        cloud = [] 
        pressure_surface = [] 
        ptype_surface = [] 
        past3hprecip_surface = [] 
        gust_surface = []

        for j in range(len(iodata[1])):
            temp.append(iodata[2][j][timePoint]) 
            uwind.append(iodata[3][j][timePoint]) 
            vwind.append(iodata[4][j][timePoint]) 
            rh.append(iodata[5][j][timePoint]) 
        for j in range(len(iodata[6])):
            cloud.append(iodata[6][j][timePoint])      
        pressure_surface.append(iodata[7][timePoint]) 
        ptype_surface.append(iodata[8][timePoint])
        past3hprecip_surface.append(iodata[9][timePoint]) 
        gust_surface.append(iodata[10][timePoint])
            
        return [timeStr, pressure, temp, uwind, vwind, rh, cloud, pressure_surface, ptype_surface, past3hprecip_surface, gust_surface]
    else:
        return ('Requested time point is out of forecast period, please check.')   

def getSurfaceWeather(iodata):
    [timeStr, pressure, temp, uwind, vwind, rh, cloud, pressure_surface, ptype_surface, past3hprecip_surface, gust_surface] = iodata
    temp_surface = temp[0]
    uwind_surface = uwind[0]
    vwind_surface = vwind[0]
    rh_surface = rh[0]
    lclouds_surface = cloud[0]
    mclouds_surface = cloud[1]
    hclouds_surface = cloud[2]
    return [timeStr, temp_surface, uwind_surface, vwind_surface, rh_surface,lclouds_surface, mclouds_surface,hclouds_surface, pressure_surface, ptype_surface, past3hprecip_surface, gust_surface]

def getSurfaceForecast(surfaceIodata):
    surface = pd.DataFrame(surfaceIodata)
    surface = surface.T
    surface.columns = ['time', 'temp_surface', 'uwind_surface', 'vwind_surface', 'rh_surface','lclouds_surface', 'mclouds_surface','hclouds_surface', 'pressure_surface', 'ptype_surface', 'past3hprecip_surface', 'gust_surface']
    def pd_round(inputSeries):
        pdd=pd.DataFrame(inputSeries)
        pdd=pdd.round(1)
        pdd=pd.Series(pdd.iloc[:,0])
        pdd=pdd.astype(int)
        return pdd
    surface['time'] = pd.to_datetime(surface['time'])
    forecast=pd.DataFrame(surface.time)
    forecast['temp_surface'] = surface['temp_surface'] - 273.15
    forecast['temp_surface'] = np.round(list(forecast['temp_surface']),decimals=1)
    forecast['precipation3h'] = surface['past3hprecip_surface']
    forecast['lclouds_show'] = pd_round(surface['lclouds_surface']/10)
    forecast['lclouds_surface'] = pd_round(surface['lclouds_surface'])
    forecast['mclouds_show'] = pd_round(surface['mclouds_surface']/10)
    forecast['mclouds_surface'] = pd_round(surface['mclouds_surface'])
    forecast['hclouds_show'] = pd_round(surface['hclouds_surface']/10)
    forecast['hclouds_surface'] = pd_round(surface['hclouds_surface'])
    forecast['wind_surface'] = np.sqrt(
        list(surface['uwind_surface']**2 + surface['vwind_surface']**2)
        )
    forecast['wind_surface'] = np.round(list(forecast['wind_surface']),decimals=1)
    forecast['gust_surface'] = np.round(list(surface['gust_surface']),decimals=1)
    forecast['rh_surface'] = pd_round(surface['rh_surface'])
    forecast['wind_aspect'] = pd_round(
        np.arctan2(
            list(surface['vwind_surface']),
            list(surface['uwind_surface'])
            )* 180 / np.pi
            )
    forecast['wind_asp']='W'
    forecast.loc[
        (forecast['wind_aspect'] == 0) |
        (forecast['wind_aspect'] == 360)
    ,'wind_asp'] = 'E'
    forecast.loc[
        (forecast['wind_aspect'] == -90)
    ,'wind_asp'] = 'S'
    forecast.loc[
        (forecast['wind_aspect'] == 180) |
        (forecast['wind_aspect'] == -180)
    ,'wind_asp'] = 'W'
    forecast.loc[
        (forecast['wind_aspect'] == 90)
    ,'wind_asp'] = 'N'
    forecast.loc[
        (forecast['wind_aspect'] > 0) &
        (forecast['wind_aspect'] < 90)
    ,'wind_asp'] = 'NE'
    forecast.loc[
        (forecast['wind_aspect'] > 90) &
        (forecast['wind_aspect'] < 180)
    ,'wind_asp'] = 'NW'
    forecast.loc[
        (forecast['wind_aspect'] > -90) &
        (forecast['wind_aspect'] < 0)
    ,'wind_asp'] = 'SE'
    forecast.loc[
        (forecast['wind_aspect'] > -180) &
        (forecast['wind_aspect'] < -90)
    ,'wind_asp'] = 'SW'
    forecast.wind_aspect = forecast['wind_asp']
    forecast['weather'] = ''
    forecast['clouds_show'] = forecast['lclouds_show']
    forecast.loc[
        (forecast['mclouds_show'] > forecast['lclouds_show']) &
        (forecast['mclouds_show'] > forecast['hclouds_show'])
    ,'clouds_show'] = list(forecast.mclouds_show[
        (forecast['mclouds_show'] > forecast['lclouds_show']) &
        (forecast['mclouds_show'] > forecast['hclouds_show'])
    ])
    forecast.loc[
        (forecast['hclouds_show'] > forecast['lclouds_show']) &
        (forecast['hclouds_show'] > forecast['mclouds_show'])
    ,'clouds_show'] = list(forecast.hclouds_show[
        (forecast['hclouds_show'] > forecast['lclouds_show']) &
        (forecast['hclouds_show'] > forecast['mclouds_show'])
    ])
    forecast.loc[
        (surface['ptype_surface'] == 0) & 
        (forecast['clouds_show'] < 2)
        ,'weather'] = '晴'
    forecast.loc[
        (surface['ptype_surface'] == 0) & 
        (forecast['clouds_show'] >= 8)
        ,'weather'] = '阴' # 参考资料: http://www.kepu.net.cn/gb/earth/weather/water/wtr010.html
    forecast.loc[
        (surface['ptype_surface'] == 0) & 
        (forecast['clouds_show'] >= 1) &
        (forecast['clouds_show'] <= 3)
        ,'weather'] = '少云' #  10%至30%称为少云
    forecast.loc[
        (surface['ptype_surface'] == 0) & 
        (forecast['clouds_show'] >= 4) &
        (forecast['clouds_show'] <= 7) 
        ,'weather'] = '多云' # 30%至70%称为多云
    forecast.loc[
        (surface['ptype_surface'] == 1)
    ,'weather'] = '雨'
    forecast.loc[
        (surface['ptype_surface'] == 5)
    ,'weather'] = '雪'
    forecast.loc[
        (surface['ptype_surface'] == 6)
    ,'weather'] = '湿雪'
    forecast = forecast.drop(['mclouds_show', 'lclouds_show', 'hclouds_show', 'clouds_show', 'wind_asp'], axis=1)
    return forecast  
    
def getNumberOfData(verticalInfo, timeStr):
    for i in range(0, len(verticalInfo[0])):
        if self.verticalInfo[0][i].find(timeStr) != -1:
            return i
    return -1

def show_forecast(forecast):
    from plotly.subplots import make_subplots
    from plotly.colors import n_colors

    def resize(data,inputX,inputY,outputX,outputY):
        scale = (outputX - outputY)/(inputX - inputY)
        datao = (data - inputY) * scale +outputY
        datao = np.round(list(datao))
        datao = np.int32(datao)
        return datao

    def temp():
        from plotly.colors import n_colors
        d1 = n_colors('rgb(102,255,255)','rgb(0,0,153)', 81, colortype='rgb')# -80 to 0
        d2 = n_colors('rgb(51,255,0)','rgb(102,255,255)', 15, colortype='rgb') # 0 to 15
        d3 = n_colors('rgb(255,255,0)','rgb(51,255,0)', 15, colortype='rgb') # 15 to 30
        d4 = n_colors('rgb(255,0,0)','rgb(255,255,0)', 20, colortype='rgb') # 30 to 50
        d = d4+d3+d2+d1
        d = d[::-1]
        return d

    timestr=forecast.time.apply(lambda x: x.strftime('%Hh %d-%m-%Y'))
    temp_colors = temp()
    cloud_colors = n_colors( 'rgb(30,144,255)', 'rgb(240,248,255)', 101, colortype='rgb')

    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.3,0.7],
        shared_xaxes=True,
        vertical_spacing=0.01,
        specs=[
            [{"secondary_y": True}],
            [{"type": "table"}]
                ]
        )
    fig.add_trace(go.Scatter(
                x = forecast.time,
                y = forecast.temp_surface,
                mode = 'lines', 
                name = 'temp(℃)',
                line = dict(color='#F08080'),
            ),row=1,col=1,secondary_y=True)
    fig.add_trace(go.Bar(
                x = forecast.time,
                y = forecast['precipation3h'],
                marker_color = '#1E90FF',
                name = '3h prec(mm)',
            ),row=1,col=1,)
    fig.add_trace(
        go.Table(
        columnorder = list(range(1,12,1)),
        columnwidth = [100,30,40,40,40,40,40,40,40,40,40],
        header=dict(
            values=[
            '<b>time</b>','',
            '<b>temp</b>', '<b>prec</b>', 
            '<b>RH</b>','<b>wind</b>','<b>aspect</b>','<b>gust</b>',
            '<b>high</b>', '<b>middle</b>', '<b>low</b>'
            ],
            line_color=['white','#D4F2E7','#D4F2E7','#D4F2E7','#D4F2E7','#FFE4C4','#FFE4C4','#FFE4C4','#B0C4DE','#B0C4DE','#B0C4DE'], 
            fill_color=['white','#D4F2E7','#D4F2E7','#D4F2E7','#D4F2E7','#FFE4C4','#FFE4C4','#FFE4C4','#B0C4DE','#B0C4DE','#B0C4DE'],
            align=['right','center','center','center','center','center','center','center','center','center'],
            font=dict(color='black', family='Times New Roman',size=12)
        ),
        cells=dict(
            values=[
            timestr,forecast.weather,
                forecast.temp_surface, np.round(list(forecast.precipation3h),decimals=3), 
                forecast.rh_surface, forecast.wind_surface, forecast.wind_aspect, forecast.gust_surface, 
                forecast.hclouds_surface, forecast.mclouds_surface, forecast.lclouds_surface
                ],
            line_color=[
                '#F5F5F5', '#F5F5F5',
                np.array(temp_colors)[np.int32(np.round(forecast.temp_surface))+79],
                '#F5F5F5', '#F5F5F5', '#F5F5F5', '#F5F5F5', '#F5F5F5', 
                np.array(cloud_colors)[forecast.hclouds_surface],
                np.array(cloud_colors)[forecast.mclouds_surface], 
                np.array(cloud_colors)[forecast.lclouds_surface]
                        ],
            fill_color=[
                'white','white',
                np.array(temp_colors)[np.int32(np.round(forecast.temp_surface))+79],
                '#A9A9A9','#A9A9A9', '#A9A9A9', '#A9A9A9','#A9A9A9',
                np.array(cloud_colors)[forecast.hclouds_surface],
                np.array(cloud_colors)[forecast.mclouds_surface], 
                np.array(cloud_colors)[forecast.lclouds_surface]
                        ],
            align=['right','center','center','center','center','center','center','center','center','center'], 
            font=dict(color='black', family='Times New Roman',size=10)
            )
        ),
        row=2,col=1
    )
    fig.update_layout(
            width=800,
            height=450,
            margin=dict(
                        l=50,
                        r=50,
                        b=20,
                        t=20,
                        pad=4
                         ),
            hovermode="x unified",
            yaxis=dict(range=[0, 10**math.ceil(math.log10(max(forecast['precipation3h'][0:20])))]),
            xaxis=dict(gridcolor='black',range=[forecast.time[0], forecast.time[20]]),
            font = dict(family='Times New Roman'),
            template='simple_white',
            showlegend=False,
            xaxis_tickformatstops = [
                dict(dtickrange=[None, 1000], value="%H:%M:%S.%L ms"),
                dict(dtickrange=[1000, 60000], value="%H:%M:%S s"),
                dict(dtickrange=[60000, 3600000], value="%H:%M m"),
                dict(dtickrange=[3600000, 86400000], value="%e %a %Hh"),
                dict(dtickrange=[86400000, 604800000], value="%e (%a)"),
                dict(dtickrange=[604800000, "M1"], value="%e. %b w"),
                dict(dtickrange=["M1", "M12"], value="%b '%y M"),
                dict(dtickrange=["M12", None], value="%Y Y")
            ],
            barmode='overlay')
    fig.update_yaxes(title_text="precipation", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="temperature", row=1, col=1, secondary_y=True)
    fig.update_xaxes(tickfont=dict(family='Times New Roman', size=1, color='black'), row=1, col=1,)
    return fig

def forecast_all(lon,lat):
    a4data = request(lon,lat)
    units,verticalInfo = getTimeSeriesVerticalWeather(a4data)
    surfaceIodata = getSurfaceWeather(verticalInfo)
    forecast = getSurfaceForecast(surfaceIodata)
    fig = show_forecast(forecast)
    return fig

def plot_kml(k, originalfig):
    coordinates = k.get_coordinates()
    fig = originalfig
    
    if k.get_type() == 'raw':  
        newfig=go.Scattermapbox(
            mode = "markers+lines",
            lat=coordinates.lat, lon=coordinates.lon, 
            hovertext=["<b>lon</b>: {lon}<br><b>lat</b>: {lat}<br><b>ele</b>: {ele}".format(lon=a,lat=b,ele=c) for a,b,c in zip(coordinates.lon, coordinates.lat, coordinates.ele)],
            hoverinfo="text",
            name = k.get_author(),
            text = "",
            meta = [coordinates.ele],
            marker = {
                'size': 4,
                })
        fig = fig.add_trace(newfig)
        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update_layout(hoverlabel=dict(
        font_family="Times Now Roman"))
        fig.update_layout(mapbox = {
                'center':go.layout.mapbox.Center(
                lat=coordinates.lat.mean(),
                lon=coordinates.lon.mean()),
                'style': "stamen-terrain",
                'zoom': k.get_tilesize()})
        
    elif k.get_type() == 'detailed':
        newfig = go.Scattermapbox(
            mode = "markers+lines",
            lat=coordinates.lat, lon=coordinates.lon, 
            hovertext=["<b>lon</b>: {lon}<br><b>lat</b>: {lat}<br><b>ele</b>: {ele}<br><b>time</b>: {time}<br><b>speed</b>: {speed}".format(lon=a,lat=b,ele=c,time=h,speed=g) for a,b,c,h,g in zip(coordinates.lon, coordinates.lat, coordinates.ele,coordinates.time,coordinates.speed)],
            hoverinfo="text",
            name=k.get_author(),
            text=coordinates.name,
            marker = {'size': 4,
                    'color': np.array(speed_colors())[k.set_colors()]
                    }
            )
        
        fig = fig.add_trace(newfig)
        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(hoverlabel=dict(
            font_family="Times Now Roman"))
        fig.update_layout(
            margin={"r":0,"t":0,"l":0,"b":0},
            legend_title_text='Author',
            ) 
        fig.update_layout(mapbox = {
                'center':go.layout.mapbox.Center(
                lat=coordinates.lat.mean(),
                lon=coordinates.lon.mean()),
                'style': "stamen-terrain",
                'zoom': k.get_tilesize()})
    return fig
    

#---------------------------------------------------------------------------------------------------------

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

init_map = set_original(121,30,8)
init_map.update_layout(clickmode='event+select')

app.layout = html.Div([
     html.H1(
        children='Tracks outside',
        style={
            'textAlign': 'center',
            'color': "#191970",            
            'background-color': "#F0F8FF"
        }
    ),
    html.Div([html.H6(
        children = 'Wanna checkout the conditions of the destination you are longing to?',
        style = {
            'textAlign': 'center',
            'color': "#191970"
        }
    )],style={
        'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center',
            'flex-direction': 'row'
    }
    ),
    html.Div([html.H6(
        children = 'Here we try to visualize kml tracks ,especially those produced by 2bulu (which do not strictly follow the standards), and see whether the weather condition is nice to go for a trip or not.',
        style = {
            'textAlign': 'center',
            'color': "#191970",

        }
    )],style={
        'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center',
            'flex-direction': 'row'
    }
    ),
    dcc.Store(id='nowkml'),
    html.Div([
        dcc.Upload(
            id='upload-data',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select Files')
            ]),
            style={
                'width': '50%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                "display": "inline-block",
                'textAlign': 'center',
                'margin-left':'25%',
                'margin-buttom':'40px' ,             
            'justify-content': 'center'
            },
        # multiple=True # Allow multiple files to be uploaded
    ),]),
    html.Div(id='output-data-upload',
            children='Infomation',
            style={
                'width': '60%',
                'borderRadius': '5px',
                'color': "#191970",
                'background-color': "#F0F8FF",
                "display": "flex",
                'textAlign': 'center',
                'margin-left':'20%',
                'margin-buttom':'20px' , 
                'margin-top':'20px',            
            'justify-content': 'center'
            },),
    # html.Div([html.Button("Download As Shapefiles", id="btn_txt"), dcc.Download(id="download-text")]),
    html.Div([dcc.Graph(
        id = 'kml-map',
        figure = init_map,style={
            'margin-top':'40px' ,
        }
            )]),
   
        html.Div([html.H6(
            id='location-clicked',
            children='Choose a location to see the weather forecast here',)],
            style={
            'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center',
            'color': "#191970",
            }), 
        html.Div([
            html.Button('Checkout Weather', id='weather-check', n_clicks=0,
        style={
            'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center',
            'color': "#191970"
        }),
        ],style={
            'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center'
        }),
    html.Div([
        html.Div([
            dcc.Graph(
                id = "weather-forecast",
                figure = init_forecast(),
                style={
                        'width':'80%',
                        "display": "flex",
                        'justify-content': 'center',
                          },
            )
        ],style={
            'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center',
        })
    ],style={
            'display': 'flex',
            'textAlign': 'center',
            'justify-content': 'center',
        }),
    dcc.Markdown(
        '''
        This is a final project for Software Engineering and GIS Development produced by Tangbo@https://github.com/Plutoset.
        '''
    ),
    # represents the URL bar, doesn't render anything
    dcc.Location(id='url', refresh=False),

    # dcc.Link('Navigate to "/"', href='/'),
])

@app.callback(
    Output('location-clicked', 'children'),
    Input('kml-map', 'clickData'))
def display_click_data(clickData):
    clickjson = json.loads(json.dumps(clickData, indent=2))
    try:
        clicklon = clickjson['points'][0]['lon']
        clicklat = clickjson['points'][0]['lat']
        outputData = dict()
        outputData['lon'] = clicklon
        outputData['lat'] = clicklat
        return u'longitude: {}, latitude: {}'.format(clicklon, clicklat)
    except:
        return 'Choose a location to see the weather forecast here'

@app.callback(
    Output('weather-forecast', 'figure'),
    [Input('weather-check','n_clicks')],
    state=[State('location-clicked', 'children')]
        )
def nowpoint_forecast(nclicks,location):
    try:
        locations = json.dumps(location)
        locations = locations.split(sep='\n')
        locations = ''.join(locations)
        locations = locations.replace('\n','')
        locations = locations.replace(' ','')
        locations = locations.replace('"','')
        locations = locations.replace("'","")
        locations = locations.replace('longitude:','')
        locations = locations.replace('latitude:','')
        locations = locations.replace('\n','')
        (lon,lat) = locations.split(sep=',')
        lon = float(lon)
        lat = float(lat)
        # forecast1 = forecast_all(lon,lat)
        import joblib
        forecast1 = joblib.load('https://github.com/Plutoset/Tracks_Outside/blob/master/forecast.pkl?raw=true')
        return forecast1
    except:
        return init_forecast()

def parse_contents(contents, filename, originalFig):
    content_type, content_string = contents.split(',')
    Fig = go.Figure(data=originalFig['data'],layout=originalFig['layout'])
    decoded = base64.b64decode(content_string)
    try:
        if '.kml' in filename:
            show_kml = kml(
                io.StringIO(decoded.decode('utf-8'))
                )
            header = show_kml.get_header()
            header = header.split(sep='<br>')
            marker = []
            for dd in header:
                if dd != '':
                    marker.append(dd)
            markers = [dcc.Markdown(i)for i in marker]
            return html.Div(
                markers
            ),show_kml.get_json(),plot_kml(show_kml,Fig)
        else:
            return html.Div([
            'Unmatching file type detected.'
        ]),'',Fig

    
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ]),'',Fig
    

@app.callback(Output('output-data-upload', 'children'),
                Output('nowkml','data'),
                Output('kml-map','figure'),
                Input('kml-map','figure'),
                Input('upload-data', 'contents'),
                State('upload-data', 'filename'),
                )
def update_output(originalFig, list_of_contents, list_of_names):
    if list_of_contents is not None:
        return parse_contents(list_of_contents, list_of_names, originalFig)
    else:
        return (None,'',originalFig)

from flask import request
server = app.server

def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    sys.exit("Bye!")

# 没想好，说不定就放弃了
# @app.callback(
#     Output("download-text", "data"),
#     Input("nowkml","data"),
#     Input("btn_txt", "n_clicks"),
#     prevent_initial_call=True,
# )
# def download(kmldata, n_clicks):
#     if kmldata != '':
#         kmlio = io.StringIO()
#         kmlio.write(kmldata)
#         download_kml = kml(kmlio)
#         return dict(content="Hello world!", filename="hello.txt")



if __name__ == '__main__':

    app.run_server(debug=False, port = 8090, use_reloader=False, dev_tools_hot_reload =True, threaded=True)
    sys.exit("Bye!")